In [1]:
!pip install pyLDAvis

In [11]:
from google.colab import drive
drive.mount('/content/drive')
PROJECT_ROOT = '/content/drive/MyDrive'

import os 
os.chdir(PROJECT_ROOT)
DATA_PATH = os.path.join(PROJECT_ROOT, 'Quotebank_limunADA')


import spacy
spacy.load('en')
from spacy.lang.en import English
parser = English()

import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

import pyLDAvis 
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()

import gensim
from gensim import corpora
import pickle
import bz2
import json

NUM_TOPICS = 10


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:126: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/local/lib/python3.7/dist-packages/catalogue.py:126: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):


In [13]:
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)

    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)

    return lda_tokens


def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    

def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)


def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens


def get_tokens_per_quote(path_to_file, print_step=5e4):

    tokens_per_quote = {}
    # Iterate through the quotes
    with bz2.open(path_to_file, 'rb') as s_file:
        for i, instance in enumerate(s_file):
            if i % int(print_step) == 0:
                print(f'Instance {i}')

            # loading a sample and checking the speaker
            instance = json.loads(instance) 
            tokens = prepare_text_for_lda(instance['quotation'])

            tokens_per_quote[instance['quoteID']] = tokens 
    
    return tokens_per_quote


def get_dictionary_and_corpus(tokens_per_quote_list):
    # List of all the words in text_data (unique, so more like a set)
    dictionary = corpora.Dictionary(tokens_per_quote_list)

    # Bag of Words representation of each of the lines 
    corpus = [dictionary.doc2bow(text) for text in tokens_per_quote_list]

    return dictionary, corpus



In [15]:
path_to_democrates = os.path.join(DATA_PATH, 'quotes-democrates-2020.json.bz2')

tokens_per_quote_democrates = get_tokens_per_quote(path_to_democrates)
tokens_per_quote_list_democrates = [
    tokens for qid, tokens in tokens_per_quote_democrates.items()
    ]

dictionary_democrates, corpus_democrates = \
    get_dictionary_and_corpus(tokens_per_quote_list_democrates)


Instance 0
Instance 50000
Instance 100000
Instance 150000
Instance 200000
Instance 250000
Instance 300000


In [19]:
path_to_republicans = os.path.join(
    DATA_PATH, 'quotes-republicans-2020.json.bz2'
    )

tokens_per_quote_republicans = get_tokens_per_quote(path_to_republicans)
tokens_per_quote_list_republicans = [
    tokens for qid, tokens in tokens_per_quote_republicans.items()
    ]

dictionary_republicans, corpus_republicans = \
    get_dictionary_and_corpus(tokens_per_quote_list_republicans)


Instance 0
Instance 50000
Instance 100000
Instance 150000
Instance 200000


In [21]:
lda_democrates = gensim.models.ldamodel.LdaModel(
    corpus_democrates, id2word=dictionary_democrates, 
    num_topics=NUM_TOPICS, passes=15
    )

topics_democrates = lda_democrates.print_topics(num_words=4)

for topic in topics_democrates:
    print(topic)

(0, '0.018*"power" + 0.014*"security" + 0.013*"would" + 0.012*"decision"')
(1, '0.093*"state" + 0.028*"government" + 0.025*"country" + 0.024*"unite"')
(2, '0.020*"family" + 0.019*"school" + 0.016*"million" + 0.015*"child"')
(3, '0.059*"people" + 0.037*"think" + 0.035*"going" + 0.033*"right"')
(4, '0.037*"would" + 0.033*"could" + 0.023*"think" + 0.020*"something"')
(5, '0.030*"health" + 0.019*"public" + 0.018*"community" + 0.014*"business"')
(6, '0.024*"campaign" + 0.022*"money" + 0.022*"party" + 0.018*"democrat"')
(7, '0.016*"people" + 0.015*"house" + 0.015*"month" + 0.013*"week"')
(8, '0.068*"president" + 0.060*"going" + 0.051*"trump" + 0.019*"american"')
(9, '0.014*"looking" + 0.012*"action" + 0.012*"forward" + 0.012*"issue"')


In [29]:
os.makedirs(os.path.join(DATA_PATH, 'LDA'), exist_ok=True)

pickle.dump(
    corpus_democrates, 
    open(os.path.join(DATA_PATH, 'LDA', 'corpus_democrates.pkl'), 'wb')
    )

pickle.dump(
    dictionary_democrates, 
    open(os.path.join(DATA_PATH, 'LDA', 'dictionary_democrates.pkl'), 'wb')
    )
    
pickle.dump(
    lda_democrates, 
    open(os.path.join(DATA_PATH, 'LDA', 'LDA_democrates.pkl'), 'wb')
    )


In [23]:
lda_republicans = gensim.models.ldamodel.LdaModel(
    corpus_republicans, id2word=dictionary_republicans, 
    num_topics=NUM_TOPICS, passes=15
    )

topics_republicans = lda_republicans.print_topics(num_words=4)

for topic in topics_republicans:
    print(topic)

(0, '0.024*"money" + 0.022*"million" + 0.020*"school" + 0.015*"everyone"')
(1, '0.074*"going" + 0.063*"think" + 0.053*"people" + 0.026*"would"')
(2, '0.028*"never" + 0.015*"anything" + 0.015*"woman" + 0.014*"child"')
(3, '0.061*"president" + 0.052*"trump" + 0.026*"american" + 0.020*"people"')
(4, '0.029*"china" + 0.017*"looking" + 0.014*"forward" + 0.012*"company"')
(5, '0.033*"state" + 0.020*"business" + 0.017*"country" + 0.014*"economy"')
(6, '0.018*"number" + 0.015*"talking" + 0.015*"medical" + 0.015*"everybody"')
(7, '0.027*"house" + 0.023*"president" + 0.019*"senate" + 0.015*"impeachment"')
(8, '0.022*"health" + 0.018*"state" + 0.017*"public" + 0.014*"protect"')
(9, '0.013*"party" + 0.013*"report" + 0.011*"fight" + 0.010*"democratic"')


In [30]:
os.makedirs(os.path.join(DATA_PATH, 'LDA'), exist_ok=True)

pickle.dump(
    corpus_republicans, 
    open(os.path.join(DATA_PATH, 'LDA', 'corpus_republicans.pkl'), 'wb')
    )

pickle.dump(
    dictionary_republicans, 
    open(os.path.join(DATA_PATH, 'LDA', 'dictionary_republicans.pkl'), 'wb')
    )
    
pickle.dump(
    lda_republicans, 
    open(os.path.join(DATA_PATH, 'LDA', 'LDA_republicans.pkl'), 'wb')
    )


## Loading pre-trained models 

In [14]:
# Democrates
corpus_democrates = pickle.load(
    open(os.path.join(DATA_PATH, 'LDA', 'corpus_democrates.pkl'), 'rb')
    )

dictionary_democrates = pickle.load(
    open(os.path.join(DATA_PATH, 'LDA', 'dictionary_democrates.pkl'), 'rb')
    )

lda_democrates = pickle.load(
    open(os.path.join(DATA_PATH, 'LDA', 'LDA_democrates.pkl'), 'rb')
    )

# Republicans
corpus_republicans = pickle.load(
    open(os.path.join(DATA_PATH, 'LDA', 'corpus_republicans.pkl'), 'rb')
    )

dictionary_republicans = pickle.load(
    open(os.path.join(DATA_PATH, 'LDA', 'dictionary_republicans.pkl'), 'rb')
    )

lda_republicans = pickle.load(
    open(os.path.join(DATA_PATH, 'LDA', 'LDA_republicans.pkl'), 'rb')
    )


In [16]:
lda_display = pyLDAvis.gensim_models.prepare(
    lda_democrates, corpus_democrates, dictionary_democrates, sort_topics=False
    )

pyLDAvis.display(lda_display)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [15]:
lda_display = pyLDAvis.gensim_models.prepare(
    lda_republicans, corpus_republicans, dictionary_republicans, sort_topics=False
    )

pyLDAvis.display(lda_display)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)
